In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
import pandas as pd
import seaborn as sns

from source.folktables.data_loader import load_folktables_data

In [3]:
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

sns.set(rc={'figure.figsize':(15, 5)})

In [4]:
try:
    from folktables import ACSDataSource, ACSEmployment
except:
    !pip install folktables
    from folktables import ACSDataSource, ACSEmployment

In [5]:
DATASET_CONFIG = {
    'state': ["LA"],
    'year': '2018',
    'task': ACSEmployment
}

In [6]:
X_data, y_data = load_folktables_data(task=DATASET_CONFIG['task'], state=DATASET_CONFIG['state'], year=DATASET_CONFIG['year'], without_nulls=True)

Original: 5 mb
Optimized: 2 mb

Dataset shape before handling nulls:  (43589, 16)
Impute values:  {'SCHL': 0, 'ESP': 0, 'MIG': 0, 'MIL': 0, 'DREM': 0}
Dataset shape after handling nulls:  (43589, 16)

Rechecking if there are nulls in X_data:
AGEP        0
SCHL        0
MAR         0
RELP        0
DIS         0
ESP         0
CIT         0
MIG         0
MIL         0
ANC         0
NATIVITY    0
DEAR        0
DEYE        0
DREM        0
SEX         0
RAC1P       0
dtype: int64


In [7]:
y_data.value_counts()

0    25363
1    18226
Name: ESR, dtype: int64

In [8]:
X_data.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P
0,29,18.0,5,16,2,0.0,1,1.0,4.0,2,1,2,2,2.0,1,1
1,17,13.0,5,16,2,0.0,1,1.0,4.0,1,1,2,2,2.0,1,2
2,37,13.0,5,16,2,0.0,1,1.0,4.0,1,1,2,2,2.0,1,1
3,86,12.0,2,16,1,0.0,1,1.0,4.0,1,1,2,1,2.0,2,2
4,22,19.0,5,17,2,0.0,1,1.0,4.0,1,1,2,2,2.0,2,1


In [9]:
y_data.head()

0    0
1    0
2    0
3    0
4    0
Name: ESR, dtype: int64

In [10]:
full_df = pd.concat([X_data, y_data], axis=1)

In [11]:
columns_to_cast = ['SCHL', 'ESP', 'MIG', 'MIL', 'DREM']
for column in columns_to_cast:
    full_df[column] = full_df[column].astype('int')

full_df.head()

,AGEP,SCHL,MAR,RELP,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,SEX,RAC1P,ESR
0,29,18,5,16,2,0,1,1,4,2,1,2,2,2,1,1,0
1,17,13,5,16,2,0,1,1,4,1,1,2,2,2,1,2,0
2,37,13,5,16,2,0,1,1,4,1,1,2,2,2,1,1,0
3,86,12,2,16,1,0,1,1,4,1,1,2,1,2,2,2,0
4,22,19,5,17,2,0,1,1,4,1,1,2,2,2,2,1,0


In [12]:
full_df['RAC1P'].value_counts()

1    29442
2    11583
9     1002
6      765
8      512
3      224
5       48
7       12
4        1
Name: RAC1P, dtype: int64

In [13]:
# Remove a class with a low number of samples to simplify training pipeline
full_df = full_df[full_df.RAC1P != 4]

In [14]:
full_df.to_csv(f'../datasets/folktables-{DATASET_CONFIG["state"][0]}-{DATASET_CONFIG["year"]}.csv', index=False)